Fundraising contact reports

IST 664

Gary Chen, Charles Cheung, & Matt Maloney

In [ ]:
## IMPORT PACKAGES
import csv
import pandas as pd
import os
import numpy as np
import nltk
import resource

In [ ]:
## CHECK COLUMNS
with open('/content/ContactReports.csv', newline='') as f:
    reader = pd.read_csv(f, delimiter=',')
    for row in reader:
        print(row)

Lookup ID
Date
Interactions\Owner\Lookup ID
Method
Category
Subcategory
Comment
PRDA Campaign Commitment Smart Field\Value
Best Rating on System Smart Field\Value


In [ ]:
## Open the file
CITRUSfile = open ('/content/ContactReports.csv', 'r')
CITRUSlist = []
with open('/content/ContactReports.csv', newline='') as f:
    reader = csv.reader(f, delimiter=',')
    for line in reader:
        ## skip lines without data
        if line[0] == '' or line[0].startswith('Lookup') or line[0].startswith('Date') or line[0].startswith('Interaction')\
                or line[0].startswith('Method') or line[0].startswith('Category') or line[0].startswith('Subcategory')\
                or line[0].startswith('Comment') or line[0].startswith('PRDA') or line[0].startswith('Best'):
            continue
        else:
            try:
                ## create a dictionary for each constituent
                donor = {}
                ## add each piece of data under a key representing that data
                donor['id'] = line[0]
                donor['date'] = line[1]
                donor['owner'] = line[2]
                donor['method'] = line[3]
                donor['cat'] = (line[4])
                donor['subcat'] = (line[5])
                donor['comment'] = (line[6])
                donor['campaign'] = (line[7])
                donor['rating'] = (line[8])
                
                ## add this donor to the list
                CITRUSlist.append(donor)
                
            except IndexError:
                print('Error: ', line)
f.close()

print("Read", len(CITRUSlist), "donor data")

Read 32951 donor data


In [ ]:
## CREATE DATAFRAME
CITRUSdf = pd.DataFrame(CITRUSlist, columns=['id','date','owner','method','cat','subcat','comment','campaign','rating'])
CITRUSdf = CITRUSdf.drop(labels=0, axis=0)
CITRUSdf

,id,date,owner,method,cat,subcat,comment,campaign,rating
1,8-10254020,1/2/2013,8-10682948,Face to Face - campus visit,Cultivation,Other,SUMMARY: Caleb is a senior at UNC Charlotte an...,$200.00,"$25,000 - $49,999"
2,8-10279366,1/3/2013,8-10267750,Face to Face - off-campus visit,Cultivation,Other,SUMMARY: Anne Globe and I met at at a hotel re...,$100.00,"$500,000 - $999,999"
3,8-10385891,1/3/2013,8-10492691,Face to Face - off-campus visit,Cultivation,Other,SUMMARY: Spencer and I met at the winter garde...,"$4,100.00","$100,000 - $249,999"
4,8-10218119,1/3/2013,8-10267750,Face to Face - off-campus visit,Stewardship,Other,SUMMARY: met with Bob at the Ironwood pizza pl...,"$1,500.00","$25,000 - $49,999"
5,8-10359642,1/3/2013,8-10222029,Face to Face - off-campus visit,Cultivation,Other,SUMMARY: Mitch brought me up to date on the la...,$0.00,"$50,000 - $99,999"
...,...,...,...,...,...,...,...,...,...
32946,8-10330347,11/21/2022,8-10224540,Face to Face - off-campus visit,Cultivation,Personal interaction,Excellent meeting with Paul today. We connect...,"$2,395.00","$100,000 - $249,999"
32947,8-10238117,11/21/2022,8-10711942,Face to Face - online visit,No Interaction Category,No Interaction Subcategory,David and I spoke for about 30 minutes to disc...,"$2,500.00","$100,000 - $249,999"
32948,8-10297598,11/21/2022,8-10658855,Face to Face - online visit,Discovery,Qualification attempt,"Met with Mike, he's been at KCBA almost 17 yea...",$950.00,"$25,000 - $49,999"
32949,8-10442295,11/21/2022,8-10448750,Face to Face - online visit,Alumni Engagement,Career/Professional Development,Resume review,$0.00,"$0 - $24,999"


In [ ]:
#lower case all the words in the comment column for evaluation
CITRUSdf['comment'] = CITRUSdf['comment'].astype(str).str.lower()

#format the comment column to remove some uncessscary string
stop_word_trim = [
'summary:',
'bio/demographic changes:',
'capital rating/inclination change:',
'corps/fdns:',
'nothing new',
'planned gift:',
'research needs:',
'stock notification:',
'solicitation/estimated gift amount:',
'professional experience:',
'family:',
'su interest/other philanthropic interest:',
'su experience/connections:',
'wealth/assets:',
'other:',
'follow-up:',
'distribution:',
'circ. list',
'optional',
'division of advancement & external affairs',
'giving.syr.edu',
'syr.edu',
'640 skytop road, 2nd floor',
'640 skytop road',
'syracuse, ny',
'sent from my iphone',
'subject:',
'recipients:']

for word in stop_word_trim:
   CITRUSdf['comment'] = CITRUSdf['comment'].str.replace(word,'')

from nltk.tokenize import RegexpTokenizer

regexp = RegexpTokenizer('\w+')
#regexp= RegexpTokenizer('\w+[:]*')

CITRUSdf['comment_token']=CITRUSdf['comment'].apply(regexp.tokenize)

display(CITRUSdf)

<ipython-input-5-bae4bf099482>:36: FutureWarning: The default value of regex will change from True to False in a future version.
  CITRUSdf['comment'] = CITRUSdf['comment'].str.replace(word,'')


,id,date,owner,method,cat,subcat,comment,campaign,rating,comment_token
1,8-10254020,1/2/2013,8-10682948,Face to Face - campus visit,Cultivation,Other,caleb is a senior at unc charlotte and is int...,$200.00,"$25,000 - $49,999","[caleb, is, a, senior, at, unc, charlotte, and..."
2,8-10279366,1/3/2013,8-10267750,Face to Face - off-campus visit,Cultivation,Other,anne globe and i met at at a hotel restaurant...,$100.00,"$500,000 - $999,999","[anne, globe, and, i, met, at, at, a, hotel, r..."
3,8-10385891,1/3/2013,8-10492691,Face to Face - off-campus visit,Cultivation,Other,spencer and i met at the winter garden at 590...,"$4,100.00","$100,000 - $249,999","[spencer, and, i, met, at, the, winter, garden..."
4,8-10218119,1/3/2013,8-10267750,Face to Face - off-campus visit,Stewardship,Other,met with bob at the ironwood pizza place in m...,"$1,500.00","$25,000 - $49,999","[met, with, bob, at, the, ironwood, pizza, pla..."
5,8-10359642,1/3/2013,8-10222029,Face to Face - off-campus visit,Cultivation,Other,mitch brought me up to date on the latest new...,$0.00,"$50,000 - $99,999","[mitch, brought, me, up, to, date, on, the, la..."
...,...,...,...,...,...,...,...,...,...,...
32946,8-10330347,11/21/2022,8-10224540,Face to Face - off-campus visit,Cultivation,Personal interaction,excellent meeting with paul today. we connect...,"$2,395.00","$100,000 - $249,999","[excellent, meeting, with, paul, today, we, co..."
32947,8-10238117,11/21/2022,8-10711942,Face to Face - online visit,No Interaction Category,No Interaction Subcategory,david and i spoke for about 30 minutes to disc...,"$2,500.00","$100,000 - $249,999","[david, and, i, spoke, for, about, 30, minutes..."
32948,8-10297598,11/21/2022,8-10658855,Face to Face - online visit,Discovery,Qualification attempt,"met with mike, he's been at kcba almost 17 yea...",$950.00,"$25,000 - $49,999","[met, with, mike, he, s, been, at, kcba, almos..."
32949,8-10442295,11/21/2022,8-10448750,Face to Face - online visit,Alumni Engagement,Career/Professional Development,resume review,$0.00,"$0 - $24,999","[resume, review]"


In [ ]:
#Create Lemmatized column for comments
import nltk
nltk.download('omw-1.4')

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

wordnet_lem = WordNetLemmatizer()

CITRUSdf['comment_lem'] = CITRUSdf['comment'].apply(wordnet_lem.lemmatize)

#Import VADER lexcion for Sentiment analysis
import nltk
nltk.download('vader_lexicon')

#Import Sentiment Intensity Analyzer
from nltk.sentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
#Use lexicon and Sentiment Intensity Analyzer to calculate polarity scores
CITRUSdf['polarity'] = CITRUSdf['comment_lem'].apply(lambda x: analyzer.polarity_scores(x))

In [ ]:
def convert_currency(val):
    """
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace(',','').replace('$', '')
    return float(new_val)



In [ ]:
CITRUSdf['campaign'] = CITRUSdf['campaign'].apply(convert_currency)

In [ ]:
## First step will be to divide the contact reports by the following groupings:
## Category: Cultivation, Discovery, Solicitation, and Stewardship
## Method: F2F (campus), F2F (off-campus), F2F (online)
## Rating: $1M+, $100k+, $25k+, other
## Campaign giving: $1M+, $100k+, $25k+, other

In [ ]:
## Rating bucket column
CITRUSdf.loc[CITRUSdf['rating'] == '$0 - $24,999', 'ratingBin'] = 'low'
CITRUSdf.loc[CITRUSdf['rating'] == 'No Rating', 'ratingBin'] = 'low'
CITRUSdf.loc[CITRUSdf['rating'] == 'Unrated', 'ratingBin'] = 'low'
CITRUSdf.loc[CITRUSdf['rating'] == '$25,000 - $49,999', 'ratingBin'] = '$25k+'
CITRUSdf.loc[CITRUSdf['rating'] == '$50,000 - $99,999', 'ratingBin'] = '$25k+'
CITRUSdf.loc[CITRUSdf['rating'] == '$100,000 - $249,999', 'ratingBin'] = '$100k+'
CITRUSdf.loc[CITRUSdf['rating'] == '$250,000 - $499,999', 'ratingBin'] = '$100k+'
CITRUSdf.loc[CITRUSdf['rating'] == '$500,000 - $999,999', 'ratingBin'] = '$100k+'
CITRUSdf.loc[CITRUSdf['rating'] == '$1,000,0000 - $2,499,999', 'ratingBin'] = '$1M+'
CITRUSdf.loc[CITRUSdf['rating'] == '$2,500,0000 - $4,999,999', 'ratingBin'] = '$1M+'
CITRUSdf.loc[CITRUSdf['rating'] == '$5,000,0000 - $9,999,999', 'ratingBin'] = '$1M+'
CITRUSdf.loc[CITRUSdf['rating'] == '$10,000,0000 - $24,999,999', 'ratingBin'] = '$1M+'
CITRUSdf.loc[CITRUSdf['rating'] == '$25,000,0000 - $49,999,999', 'ratingBin'] = '$1M+'
CITRUSdf.loc[CITRUSdf['rating'] == '$50,000,0000 - $99,999,999', 'ratingBin'] = '$1M+'

In [ ]:
## GIVING
#updated bins with starting -1 
CITRUSdf['campaignBin'] = pd.cut(CITRUSdf['campaign'], [-1, 25000, 100000, 1000000, 100000000], labels=['givingLow', 'giving25k', 'giving100k','giving1M'])

In [ ]:
## CATEGORIES
cultivation = CITRUSdf[CITRUSdf['cat']=='Cultivation']
discovery = CITRUSdf[CITRUSdf['cat']=='Discovery']
solicitation = CITRUSdf[CITRUSdf['cat']=='Solicitation']
stewardship = CITRUSdf[CITRUSdf['cat']=='Stewardship']

## METHOD
campus = CITRUSdf[CITRUSdf['method']=='Face to Face - campus visit']
elsewhere = CITRUSdf[CITRUSdf['method']=='Face to Face - off-campus visit']
online = CITRUSdf[CITRUSdf['method']=='Face to Face - online']

In [ ]:
## RATING
ratingLow =  CITRUSdf[CITRUSdf['ratingBin']=='low']
rating25k =  CITRUSdf[CITRUSdf['ratingBin']=='$25k+']
rating100k = CITRUSdf[CITRUSdf['ratingBin']=='$100k+']
rating1M =  CITRUSdf[CITRUSdf['ratingBin']=='$1M+']

In [ ]:
## GIVING
## greater than $1M
giving1M = CITRUSdf[CITRUSdf['campaign']>999999]
## greater than $100k
## less than $1M
giving100k = CITRUSdf[CITRUSdf['campaign']>99999]
giving100k = giving100k[giving100k['campaign']<1000000]
## greater than $25k
## less than $100k
giving25k = CITRUSdf[CITRUSdf['campaign']<100000]
## less than $25k
givingLow = giving25k[giving25k['campaign']<25000]
## cut off low end for $25k
giving25k = giving25k[giving25k['campaign']>24999]

In [ ]:
## MAJOR GIVING - CUT AT $100k MARK
CITRUSdf['majorGiving'] = pd.cut(CITRUSdf['campaign'], [-1,100000,10000000000000], labels=['low','high'])

In [ ]:
## ONLY COMMENT FIELD
cultivation = cultivation['comment']
discovery = discovery['comment']
solicitation = solicitation['comment']
stewardship = stewardship['comment']

ratingLow =  ratingLow['comment']
rating25k =  rating25k['comment']
rating100k = rating100k['comment']
rating1M =  rating1M['comment']

campus = campus['comment']
elsewhere = elsewhere['comment']
online = online['comment']

giving1M = giving1M['comment']
giving100k = giving100k['comment']
giving25k = giving25k['comment']
givingLow = givingLow['comment']

In [ ]:
#Create new DF view with Polarity scores split into seperate columns
df_pol = pd.concat(
    [CITRUSdf.drop(['id','date','owner'], axis =1),
    CITRUSdf['polarity'].apply(pd.Series)], axis=1)


#Create variable for neutral, positve, and negative sentiment reads
#Do it based on compound score  
#https://www.kirenz.com/post/2021-12-11-text-mining-and-sentiment-analysis-with-nltk-and-pandas-in-python/text-mining-and-sentiment-analysis-with-nltk-and-pandas-in-python/

df_pol['sentiment'] = df_pol['compound'].apply(lambda x: 'positive' if x >0 else 'neutral' if x==0 else 'negative')

#for score in df_pol['compound']:
#  if score > 0.0:
#    df_pol['sentiment'] = 'positive'
#  if score == 0.0:
#    df_pol['sentiment'] = 'neutral'
#  else:
#    df_pol['sentiment'] = 'negative'

display(df_pol)

,method,cat,subcat,comment,campaign,rating,comment_token,comment_lem,polarity,ratingBin,campaignBin,majorGiving,neg,neu,pos,compound,sentiment
1,Face to Face - campus visit,Cultivation,Other,caleb is a senior at unc charlotte and is int...,200.0,"$25,000 - $49,999","[caleb, is, a, senior, at, unc, charlotte, and...",caleb is a senior at unc charlotte and is int...,"{'neg': 0.0, 'neu': 0.822, 'pos': 0.178, 'comp...",$25k+,givingLow,low,0.000,0.822,0.178,0.6597,positive
2,Face to Face - off-campus visit,Cultivation,Other,anne globe and i met at at a hotel restaurant...,100.0,"$500,000 - $999,999","[anne, globe, and, i, met, at, at, a, hotel, r...",anne globe and i met at at a hotel restaurant...,"{'neg': 0.0, 'neu': 0.928, 'pos': 0.072, 'comp...",$100k+,givingLow,low,0.000,0.928,0.072,0.6249,positive
3,Face to Face - off-campus visit,Cultivation,Other,spencer and i met at the winter garden at 590...,4100.0,"$100,000 - $249,999","[spencer, and, i, met, at, the, winter, garden...",spencer and i met at the winter garden at 590...,"{'neg': 0.043, 'neu': 0.819, 'pos': 0.138, 'co...",$100k+,givingLow,low,0.043,0.819,0.138,0.9554,positive
4,Face to Face - off-campus visit,Stewardship,Other,met with bob at the ironwood pizza place in m...,1500.0,"$25,000 - $49,999","[met, with, bob, at, the, ironwood, pizza, pla...",met with bob at the ironwood pizza place in m...,"{'neg': 0.0, 'neu': 0.905, 'pos': 0.095, 'comp...",$25k+,givingLow,low,0.000,0.905,0.095,0.7713,positive
5,Face to Face - off-campus visit,Cultivation,Other,mitch brought me up to date on the latest new...,0.0,"$50,000 - $99,999","[mitch, brought, me, up, to, date, on, the, la...",mitch brought me up to date on the latest new...,"{'neg': 0.0, 'neu': 0.717, 'pos': 0.283, 'comp...",$25k+,givingLow,low,0.000,0.717,0.283,0.9468,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32946,Face to Face - off-campus visit,Cultivation,Personal interaction,excellent meeting with paul today. we connect...,2395.0,"$100,000 - $249,999","[excellent, meeting, with, paul, today, we, co...",excellent meeting with paul today. we connect...,"{'neg': 0.023, 'neu': 0.829, 'pos': 0.148, 'co...",$100k+,givingLow,low,0.023,0.829,0.148,0.9825,positive
32947,Face to Face - online visit,No Interaction Category,No Interaction Subcategory,david and i spoke for about 30 minutes to disc...,2500.0,"$100,000 - $249,999","[david, and, i, spoke, for, about, 30, minutes...",david and i spoke for about 30 minutes to disc...,"{'neg': 0.0, 'neu': 0.927, 'pos': 0.073, 'comp...",$100k+,givingLow,low,0.000,0.927,0.073,0.8462,positive
32948,Face to Face - online visit,Discovery,Qualification attempt,"met with mike, he's been at kcba almost 17 yea...",950.0,"$25,000 - $49,999","[met, with, mike, he, s, been, at, kcba, almos...","met with mike, he's been at kcba almost 17 yea...","{'neg': 0.0, 'neu': 0.907, 'pos': 0.093, 'comp...",$25k+,givingLow,low,0.000,0.907,0.093,0.9485,positive
32949,Face to Face - online visit,Alumni Engagement,Career/Professional Development,resume review,0.0,"$0 - $24,999","[resume, review]",resume review,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",low,givingLow,low,0.000,1.000,0.000,0.0000,neutral


In [ ]:
#Polarity average calculations by campaignBin
pol = df_pol.groupby(['campaignBin']).mean()
display(pol)
print('\n')

#Polarity avg cal by ratingBin
pol_2 = df_pol.groupby(['ratingBin']).mean()
display(pol_2)
print('\n')

#Polarity avg cal by majorGiving bins
pol_3 = df_pol.groupby(['majorGiving']).mean()
display(pol_3)

,campaign,neg,neu,pos,compound
campaignBin,,,,,
givingLow,3.320431e+03,0.014309,0.860467,0.125224,0.719883
giving25k,5.235523e+04,0.012375,0.859714,0.127915,0.724331
giving100k,3.026764e+05,0.014880,0.852315,0.132590,0.740399
giving1M,4.528453e+06,0.014379,0.859303,0.126321,0.700582


,campaign,neg,neu,pos,compound
ratingBin,,,,,
$100k+,165313.428488,0.013905,0.857518,0.128475,0.743636
$25k+,51535.867809,0.015619,0.856655,0.127727,0.738488
low,28818.075657,0.012069,0.867138,0.120792,0.649069


,campaign,neg,neu,pos,compound
majorGiving,,,,,
low,1.179724e+04,0.013975,0.860336,0.125689,0.720652
high,1.524295e+06,0.014735,0.854335,0.130778,0.728888


In [ ]:
print('Overall Sentiment')
unique_sentiment = df_pol['sentiment'].value_counts()
display(unique_sentiment)

Overall Sentiment


positive    28851
neutral      3199
negative      900
Name: sentiment, dtype: int64

In [ ]:
majorLow_sent =  df_pol[df_pol['majorGiving']=='low']
print('Low MajorGiving Sentiment')
display(majorLow_sent['sentiment'].value_counts())

print('\n')

print('High MajorGiving Sentiment')
majorhigh_sent =  df_pol[df_pol['majorGiving']=='high']
display(majorhigh_sent['sentiment'].value_counts())

Low MajorGiving Sentiment


positive    23060
neutral      2587
negative      696
Name: sentiment, dtype: int64



High MajorGiving Sentiment


positive    5791
neutral      612
negative     204
Name: sentiment, dtype: int64

In [ ]:
nltk.download('punkt')
from nltk import FreqDist

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
CITRUSdf['comment_token']

1        [caleb, is, a, senior, at, unc, charlotte, and...
2        [anne, globe, and, i, met, at, at, a, hotel, r...
3        [spencer, and, i, met, at, the, winter, garden...
4        [met, with, bob, at, the, ironwood, pizza, pla...
5        [mitch, brought, me, up, to, date, on, the, la...
                               ...                        
32946    [excellent, meeting, with, paul, today, we, co...
32947    [david, and, i, spoke, for, about, 30, minutes...
32948    [met, with, mike, he, s, been, at, kcba, almos...
32949                                     [resume, review]
32950    [had, lunch, with, kwp, to, introduce, her, to...
Name: comment_token, Length: 32950, dtype: object

In [ ]:
#all_words will store all of the words/token split into one list 
all_words = []

#Bring all comment sentence tokens into 1 list/variable
#list of list of comment sentences
#Sentence tokenized
all_words_list = CITRUSdf['comment_token'].values.tolist()

for sent in all_words_list: #CITRUSdf['comment_token']:
  for word in sent:
    all_words.append(word)

#see formating is a list of all the comment tokens
print(all_words[:10])

#see formating is a list of list with sentences
#each sub-list is a sentence/row entry of comment tokens
print(all_words_list[:10])



['caleb', 'is', 'a', 'senior', 'at', 'unc', 'charlotte', 'and', 'is', 'interested']
[['caleb', 'is', 'a', 'senior', 'at', 'unc', 'charlotte', 'and', 'is', 'interested', 'in', 'attending', 'law', 'school', 'in', 'addition', 'to', 'sucol', 'caleb', 'is', 'also', 'interested', 'in', 'the', 'charlotte', 'school', 'of', 'law'], ['anne', 'globe', 'and', 'i', 'met', 'at', 'at', 'a', 'hotel', 'restaurant', 'in', 'la', 'we', 'spoke', 'about', 'her', 'experience', 'and', 'su', 'with', 'jeffery', 'katzenberg', 'and', 'her', 'own', 'time', 'here', 'the', 'substantive', 'part', 'of', 'the', 'meeting', 'dealt', 'with', 'addressing', 'my', 'proposal', 'to', 'her', 'for', 'personal', 'giving', 'and', 'to', 'help', 'us', 'make', 'inroads', 'to', 'dreamworks', 'university', 'relations', 'program', 'however', 'she', 'had', 'apparently', 'not', 'reviewed', 'the', 'materials', 'and', 'wanted', 'me', 'to', 'resubmit', 'them', 'to', 'her'], ['spencer', 'and', 'i', 'met', 'at', 'the', 'winter', 'garden', 'at'

In [ ]:
#all_comments brings all of the comments in our dataset into 1 long string
all_comments = ' '.join([word for word in CITRUSdf['comment']])

#tokenized_words = nltk.tokenize.word_tokenize(all_words)



print(all_comments[:100])

 caleb is a senior at unc charlotte and is interested in attending law school.  in addition to sucol


In [ ]:
all_words_2 = ' '.join([word for word in CITRUSdf['comment']])
#tokenized_words = nltk.tokenize.word_tokenize(all_words)

print(all_words_2)

#fdist = FreqDist(tokenized_words)
#fdist = FreqDist(all_words_list)

#Frequency Distribution of all the words
fdist = FreqDist(all_words)
fdist_50 = fdist.most_common(50)
for item in fdist_50:
  print(item)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



('the', 200912)
('to', 188457)
('and', 185198)
('a', 118825)
('in', 110589)
('he', 106588)
('of', 82835)
('with', 74326)
('for', 73649)
('is', 69188)
('i', 60046)
('his', 59665)
('that', 56153)
('she', 52700)
('at', 48941)
('was', 44209)
('on', 41531)
('her', 41249)
('we', 33464)
('s', 33275)
('about', 32485)
('as', 32223)
('be', 30244)
('will', 28210)
('it', 25540)
('they', 25017)
('has', 24484)
('su', 23948)
('him', 23760)
('would', 22675)
('had', 20923)
('but', 20434)
('this', 19755)
('up', 19606)
('from', 19185)
('an', 18845)
('not', 16906)
('have', 16822)
('gift', 16665)
('time', 16453)
('met', 16349)
('also', 15756)
('are', 15725)
('very', 14682)
('their', 14452)
('said', 13031)
('me', 12402)
('our', 11992)
('meeting', 11386)
('students', 11237)


In [ ]:
#bring in stop words
nltk.download('stopwords')
nltkstopwords = nltk.corpus.stopwords.words('english')
morestopwords = ['could','would','might','must','need','sha','wo','y',"'s","'d","'ll","'t","'m","'re","'ve", "n't"]
stopwords = nltkstopwords + morestopwords




#print(len(all_words))
#print(len(stopwords))

#filter out the stop words from our all_words list 
stoppedX = []
for word in all_words:
  if word not in stopwords:
    stoppedX.append(word)

print(len(stoppedX))

#Frequency Distribution with stopped words removed
fdist_stop_trim = FreqDist(stoppedX)
fdist_stop_50 = fdist_stop_trim.most_common(50)
for item in fdist_stop_50:
  print(item)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


2728093
('su', 23948)
('gift', 16665)
('time', 16453)
('met', 16349)
('also', 15756)
('said', 13031)
('meeting', 11386)
('students', 11237)
('year', 11181)
('school', 10672)
('campus', 10570)
('syracuse', 10524)
('years', 10149)
('asked', 10079)
('well', 9860)
('new', 9700)
('work', 9636)
('next', 9229)
('like', 9136)
('alumni', 8938)
('back', 8916)
('program', 8686)
('visit', 8506)
('university', 8364)
('support', 8320)
('dean', 8258)
('one', 7980)
('get', 7952)
('interested', 7285)
('follow', 7281)
('family', 7068)
('talked', 6894)
('college', 6876)
('board', 6668)
('great', 6376)
('shared', 6354)
('giving', 6340)
('see', 6264)
('mentioned', 6177)
('us', 6055)
('make', 5965)
('good', 5875)
('told', 5762)
('student', 5732)
('john', 5688)
('home', 5620)
('lunch', 5592)
('office', 5525)
('career', 5509)
('conversation', 5500)


In [ ]:
# get the 1500 most frequently appearing keywords in the corpus
word_items = fdist.most_common(1500)
word_features = [word for (word,count) in word_items]

In [ ]:
# after all data fixes, build the list of tuples for the feature function
CITRUSdf_cb = CITRUSdf[['comment_token','campaignBin']]
CITRUSlist = list(CITRUSdf_cb.itertuples(index=False, name=None))

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
## POS Tagging, to retrieve adjective (JJs) and adverb (RBs) tags

# use the Stanford POS tagger to POS tag tokens of each sentence
# this is the default tagger in nltk
taggedText = [nltk.pos_tag(tokens) for tokens in all_words_list]
print(taggedText[:2])

[[('caleb', 'NN'), ('is', 'VBZ'), ('a', 'DT'), ('senior', 'JJ'), ('at', 'IN'), ('unc', 'JJ'), ('charlotte', 'NN'), ('and', 'CC'), ('is', 'VBZ'), ('interested', 'JJ'), ('in', 'IN'), ('attending', 'VBG'), ('law', 'NN'), ('school', 'NN'), ('in', 'IN'), ('addition', 'NN'), ('to', 'TO'), ('sucol', 'VB'), ('caleb', 'NN'), ('is', 'VBZ'), ('also', 'RB'), ('interested', 'JJ'), ('in', 'IN'), ('the', 'DT'), ('charlotte', 'JJ'), ('school', 'NN'), ('of', 'IN'), ('law', 'NN')], [('anne', 'NN'), ('globe', 'NN'), ('and', 'CC'), ('i', 'JJ'), ('met', 'VBD'), ('at', 'IN'), ('at', 'IN'), ('a', 'DT'), ('hotel', 'NN'), ('restaurant', 'NN'), ('in', 'IN'), ('la', 'NN'), ('we', 'PRP'), ('spoke', 'VBD'), ('about', 'IN'), ('her', 'PRP$'), ('experience', 'NN'), ('and', 'CC'), ('su', 'NN'), ('with', 'IN'), ('jeffery', 'NN'), ('katzenberg', 'NN'), ('and', 'CC'), ('her', 'PRP$'), ('own', 'JJ'), ('time', 'NN'), ('here', 'RB'), ('the', 'DT'), ('substantive', 'JJ'), ('part', 'NN'), ('of', 'IN'), ('the', 'DT'), ('meetin

In [ ]:
# Following our NLTK textbook, chapter on Information Extraction--Chunking (https://www.nltk.org/book/ch07.html)

# Using CHUNKING to parse sentences 
# to look for "adjective phrases", i.e. phrases (or chunks) that have adverbs and adjectives ('RB'+'JJ')
# First step: writing a grammar that defines the POS in the chunk
# we name this grammar "ADJPH" ("ADJective PHrase") using regexes 

import re
grammar_adjph = "ADJPH: {<RB.?>+<JJ.?>}"
# This regex reads as: "find groups ("< >") of RBs (adverbs) together with groups of JJs (adjectives), with groups defineds as
# RBs with any ending (the "." is a placeholder or wildcard for the "R" and the "S" at the end of RBR and RBS, 
# while "?" indicates "optional character" so RB can be found alone as well). Same regex operators apply to JJs.

# Second step: import the nltk parser to process each sentence
chunk_parser_adj = nltk.RegexpParser(grammar_adjph)

adjph_tags = []
for sent in taggedText:
    if len(sent) > 0:
        tree = chunk_parser_adj.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'ADJPH':
                adjph_tags.append(subtree)
                
# Visualizing the actual adjective phrase
adjective_phrases = []
for sent in adjph_tags:
    temp = ''
    for w, t in sent:
        temp += w+ ' '    
    adjective_phrases.append(temp)
    
print('First 10 adjective phrases: ', adjective_phrases[:10])


# Following our NLTK textbook, chapter 1 on Language Processing (https://www.nltk.org/book/ch01.html)

## FREQUENCY DISTRIBUTIONS
# Top 50 adjective phrases
freq_adjph = nltk.FreqDist(adjective_phrases)

print('Top adjective phrases by frequency: ')
for word, freq in freq_adjph.most_common(50):
    print(word, freq)

            
#print the list of our sentences:
print('Length of adjective phrase sentences: ', len(adjph_tags))

First 10 adjective phrases:  ['also interested ', 'very appreciative ', 'very successful ', 'very helpful ', 'very helpful ', 'alum tiffany ', 'together su ', 'very productive ', 'very polished ', 'very willing ']
Top adjective phrases by frequency: 
very interested  1041
not sure  586
very happy  511
very excited  509
very pleased  440
very good  426
very much  395
so i  352
not interested  349
very nice  348
so much  347
very proud  303
very positive  295
very impressed  293
very busy  291
very involved  287
not able  245
also interested  244
too much  231
so many  206
very successful  204
very active  204
very appreciative  199
very passionate  186
more involved  179
most likely  168
most recent  163
very close  159
more engaged  152
very grateful  146
not ready  140
very important  134
very interesting  126
very enthusiastic  124
very helpful  119
lally complex  108
most interested  101
more active  99
very fond  97
very pleasant  95
more interested  89
much more  89
as much  86
ve

In [ ]:
# Now we look for "adverb phrases" or chunks that have 2 consecutive adverbs ('RB')
# First step: writing a grammar that defines POS rules of the adverb phrase the chunk
# we name this grammar "ADVPH" ("ADVerb PHrase")
grammar_advph = "ADVPH: {<RB>+<RB>}"

# Second step: import the nltk parser to process each sentence
chunk_parser_adv = nltk.RegexpParser(grammar_advph)

advph_tags = []
for sent in taggedText:
    if len(sent) > 0:
        tree = chunk_parser_adv.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'ADVPH':
                advph_tags.append(subtree)
                
# Visualizing the actual adjective phrase
adverb_phrases = []
for sent in advph_tags:
    temp = ''
    for w, t in sent:
        temp += w+ ' '    
    adverb_phrases.append(temp)
    
print('First 10 adverb phrases: ', adverb_phrases[:10])

# top 50 adjective phrases
freq_advph = nltk.FreqDist(adverb_phrases)

print('Top adverb phrases by frequency: ')
for word, freq in freq_advph.most_common(50):
    print(word, freq)

            
#print the list of our sentences:
print('Length of adverb phrase sentences: ', len(advph_tags))

First 10 adverb phrases:  ['apparently not ', 'together also ', 'together again ', 'additionally isaac ', 'no longer ', 'well however ', 'right now ', 'as well ', 'not necessarily ', 'phenomenally well ']
Top adverb phrases by frequency: 
as well  3223
very well  867
right now  707
very much  365
not yet  216
still very  174
not only  172
as soon  153
so far  146
also very  139
again soon  138
very close  132
so i  126
as long  117
not currently  108
not just  106
once again  105
so much  105
no longer  98
not really  98
just recently  96
quite well  94
very highly  92
as far  91
probably not  71
not very  71
as much  69
still not  66
well i  62
really well  62
not too  61
there as well  58
not even  57
right away  56
extremely well  54
so well  52
not necessarily  50
not so  50
not as  50
thus far  49
just not  48
anytime soon  48
not quite  42
together again  40
never really  39
currently not  39
very soon  38
so very  37
however i  35
clearly very  35
Length of adverb phrase sentenc

In [ ]:
# Top 50 adjective tokens

adjective_tokens = []
for sentence in taggedText:
    for word, pos in sentence:
        if pos in ['JJ', 'JJR', 'JJS']: # adjective, comparative, superlative
            if len(word)>1:
                adjective_tokens.append(word)
freq_adjective = nltk.FreqDist(adjective_tokens)

for word, freq in freq_adjective.most_common(50):
    print(word,freq)

new 9700
next 8620
interested 7210
other 6648
more 6571
great 6376
good 5849
su 5178
annual 4715
happy 4422
many 4196
few 4194
first 3961
major 3705
last 3695
able 3579
dean 3541
current 2810
much 2739
high 2586
several 2374
alumni 2268
long 2109
sure 2103
financial 2069
best 1932
recent 1929
young 1872
possible 1856
real 1847
old 1793
willing 1790
nyc 1754
own 1708
local 1698
small 1654
public 1654
due 1640
senior 1636
nice 1551
regional 1543
advisory 1526
open 1518
involved 1507
important 1474
future 1434
big 1429
same 1410
past 1390
whitman 1386


In [ ]:
adverb_tokens = []
for sentence in taggedText:
    for word, pos in sentence:
        if pos in ['RB', 'RBR', 'RBS']:
            if len(word)>1:
                adverb_tokens.append(word)
freq_adverb = nltk.FreqDist(adverb_tokens)

for word, freq in freq_adverb.most_common(50):
    print(word,freq)

not 16906
also 15756
very 14666
well 9732
back 7922
now 7247
so 7215
still 5103
then 5001
just 4867
as 4555
more 4343
again 3398
really 3328
forward 3282
currently 3035
there 2939
recently 2915
together 2596
here 2364
up 2118
only 1844
much 1836
however 1765
soon 1746
too 1608
most 1538
later 1524
quite 1504
even 1501
once 1495
always 1466
ago 1418
away 1376
never 1237
especially 1168
right 1137
prior 1127
yet 1065
already 1048
originally 1016
perhaps 934
clearly 901
ever 859
specifically 850
first 845
abroad 793
about 740
possibly 720
often 714


In [ ]:
# Top 50 verb tokens

verb_tokens = []
for sentence in taggedText:
    for word, pos in sentence:
        if pos in ['VB', 'VBD', 'VBG','VBN','VBP','VBZ']:
            if len(word)>1:
                verb_tokens.append(word)
freq_verb = nltk.FreqDist(verb_tokens)

for word, freq in freq_verb.most_common(50):
    print(word,freq)

is 69188
was 44209
be 30244
has 24484
had 20920
have 16808
met 15877
are 15725
said 13031
asked 10043
been 8716
do 7765
were 7734
get 7705
talked 6864
did 6473
follow 6433
giving 6340
shared 6333
mentioned 6162
see 6039
make 5908
told 5743
doing 5599
working 5337
like 5202
discussed 5041
going 5038
meet 4812
know 4534
went 4404
give 4300
help 4219
being 4137
su 4020
discuss 3878
attended 3663
send 3629
am 3612
continue 3547
go 3345
consider 3306
wants 3290
wanted 3264
involved 3229
does 3181
made 3176
looking 3158
worked 3148
work 3136


In [ ]:
# Top 50 noun tokens
noun_tokens = []
for sentence in taggedText:
    for word, pos in sentence:
        if pos in ['NN', 'NNS', 'NNP','NNPS']:
            if len(word)>1:
                noun_tokens.append(word)
freq_noun = nltk.FreqDist(noun_tokens)

for word, freq in freq_noun.most_common(50):
    print(word,freq)

gift 16460
time 16453
su 14435
students 11237
year 11181
school 10672
years 10149
meeting 9838
program 8686
campus 8272
university 8202
syracuse 7809
family 7068
college 6876
board 6668
work 6499
visit 6438
alumni 5960
student 5581
office 5525
home 5495
event 5494
conversation 5485
law 5380
lunch 5348
career 5333
support 5298
wife 5281
business 5017
scholarship 4988
son 4901
fund 4895
john 4735
interest 4700
area 4511
daughter 4499
estate 4288
dean 4176
dinner 4050
none 4014
company 3979
job 3797
experience 3731
information 3702
children 3613
maxwell 3610
part 3221
day 3085
steps 2976
opportunity 2959


In [ ]:
# Now we have two lists of POS tags combinations we can compare
# We need to get the sentences back from the tagging exercise and run some stats

# Create a list of original sentences from the ADJECTIVE phrase subset:
adjph_whole_sentences = []

# loop over the sentences in the adjective phrase sentences we created:
for sents in adjph_tags:
    temp=''
    for (word,tag) in sents:
        temp += word+' '
        adjph_whole_sentences.append(temp)
        
print(len(adjph_whole_sentences))

74459


In [ ]:
# Create a list of original sentences from the ADVERB phrase subset:
advph_whole_sentences = []

# loop over the sentences in the adjective phrase sentences we created:
for sents in advph_tags:
    temp=''
    for (word,tag) in sents:
        temp += word+' '
        advph_whole_sentences.append(temp)
        
print(len(advph_whole_sentences))

38117


In [ ]:
# OPTIONAL STEP: Combine lists together to have a single list of adjective/adverb phrases:
# Useful to know which sentences are heavy in qualifiers

# create a new variable to store all adjective phrase sentences
adv_adj_phrase_sentences = adjph_whole_sentences

# iterate over adverb phrase sentences
for sent in advph_whole_sentences:
    # if a sentence is not in the adjective phrases list imported
    if sent not in adv_adj_phrase_sentences:
        # attach that sentence
        adv_adj_phrase_sentences.append(sent)

# print the lenght of the list (i.e. number of sentences with both adjective and adverb phrases)
print(len(adv_adj_phrase_sentences))

78617


In [ ]:
# Following our NLTK textbook, Writing Structural Programs chapter
# section on Procedural vs Declarative style (http://www.nltk.org/book_1ed/ch04.html) 

## CORPUS STATISTICS--SENTENCES LENGTH

# Calculating the average length of sentences in the entire corpus
# from http://www.nltk.org/book_1ed/ch04.html
total_corpus = sum(len(sent) for sent in all_words_list) # remember: 'textsplit' is our text split into sentences
print(total_corpus / len(all_words_list))

159.43040971168438


In [ ]:
# Calculate the average length of an adjective phrase sentence
# We can then compare the average length of the adjective phrases to 
# the average sentences we calculated for all sentences in the corpus
total_adjph_sentences = sum(len(sent) for sent in adjph_whole_sentences) # adjph_whole_sentences stores our adjective phrases
print(total_adjph_sentences / len(adjph_whole_sentences))

9.8713764198583


In [ ]:
# Demonstrate feature function: document_features
# feature data set: comment_token and majorGiving

import random
# after all data fixes, build the list of tuples for the feature function
CITRUSdf_cb = CITRUSdf[['comment_token','majorGiving']]

CITRUSlist = list(CITRUSdf_cb.itertuples(index=False, name=None))

# random.shuffle(CITRUSlist) 

# get all words from all comments and put into a frequency distribution 
#   note lowercase, but no stemming or stopwords
all_words_list = [word for (sent,cat) in CITRUSlist for word in sent]
all_words = nltk.FreqDist(all_words_list)

# get the 2000 most frequently appearing keywords in the corpus 
word_items = all_words.most_common(2000)
word_features = [word for (word,count) in word_items]

# define features (keywords) of a document for a BOW/unigram baseline 
# each feature is 'contains(keyword)' and is true or false depending
# on whether that keyword is in the document
def document_features(document, word_features):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['V_{}'.format(word)] = (word in document_words)
    return features

# get features sets for a document, including keyword features and category feature
featuresets = [(document_features(d, word_features), c) for (d, c) in CITRUSlist]

# training using naive Baysian classifier, training set is approximately 50% of data 
train_set, test_set = featuresets[15000:], featuresets[:15000]
classifier = nltk.NaiveBayesClassifier.train(train_set)

# the accuracy result may vary since we randomized the documents 
# show which features of classifier are most informative
classifier.show_most_informative_features(50)


Most Informative Features
                   V_lag = True              low : high   =     45.6 : 1.0
          V_disqualified = True              low : high   =     29.6 : 1.0
                   V_ths = True              low : high   =     13.9 : 1.0
           V_inclination = True              low : high   =     12.4 : 1.0
                 V_molen = True             high : low    =      8.8 : 1.0
                   V_ter = True             high : low    =      8.8 : 1.0
                   V_wsm = True             high : low    =      8.4 : 1.0
                    V_mg = True              low : high   =      8.1 : 1.0
                V_mutual = True              low : high   =      7.8 : 1.0
            V_mentorship = True              low : high   =      7.7 : 1.0
                  V_dick = True             high : low    =      7.1 : 1.0
                   V_bot = True             high : low    =      7.1 : 1.0
               V_syverud = True             high : low    =      7.0 : 1.0

In [ ]:
# evaluate the accuracy of the classifier
nltk.classify.accuracy(classifier, test_set)

0.7308666666666667

In [ ]:
unique_rating = CITRUSdf['majorGiving'].value_counts()

In [ ]:
unique_rating

low     26343
high     6607
Name: majorGiving, dtype: int64

In [ ]:
## other evaluation measures:  confusion matrix, precision, recall, F1 ##

goldlist = []
predictedlist = []
for (features, label) in test_set:
    	goldlist.append(label)
    	predictedlist.append(classifier.classify(features))

# look at the first 30 examples
print(goldlist[:30])
print(predictedlist[:30])

['low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'high', 'low', 'low', 'low', 'low', 'low', 'high', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low']
['low', 'low', 'high', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'high', 'low', 'high', 'low', 'low', 'high', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'low', 'high', 'low']


In [ ]:
cm = nltk.ConfusionMatrix(goldlist, predictedlist)
print(cm.pretty_format(sort_by_count=True, truncate=9))

# or show the results as percentages
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=9))


     |         h |
     |    l    i |
     |    o    g |
     |    w    h |
-----+-----------+
 low |<9392>2528 |
high | 1509<1571>|
-----+-----------+
(row = reference; col = test)

     |             h |
     |      l      i |
     |      o      g |
     |      w      h |
-----+---------------+
 low | <62.6%> 16.9% |
high |  10.1% <10.5%>|
-----+---------------+
(row = reference; col = test)



In [ ]:
# Function to compute precision, recall and F1 for each label
#  and for any number of labels
# Input: list of gold labels, list of predicted labels (in same order)
# Output:  prints precision, recall and F1 for each label
def eval_measures(gold, predicted):
    # get a list of labels
    labels = list(set(gold))
    # these lists have values for each label 
    recall_list = []
    precision_list = []
    F1_list = []
    for lab in labels:
        # for each label, compare gold and predicted lists and compute values
        TP = FP = FN = TN = 0
        for i, val in enumerate(gold):
            if val == lab and predicted[i] == lab:  TP += 1
            if val == lab and predicted[i] != lab:  FN += 1
            if val != lab and predicted[i] == lab:  FP += 1
            if val != lab and predicted[i] != lab:  TN += 1
        # use these to compute recall, precision, F1
        recall = TP / (TP + FP)
        precision = TP / (TP + FN)
        recall_list.append(recall)
        precision_list.append(precision)
        F1_list.append( 2 * (recall * precision) / (recall + precision))

    # the evaluation measures in a table with one row per label
    print('\tPrecision\tRecall\t\tF1')
    # print measures for each label
    for i, lab in enumerate(labels):
        print(lab, '\t', "{:10.3f}".format(precision_list[i]), \
          "{:10.3f}".format(recall_list[i]), "{:10.3f}".format(F1_list[i]))

# call the function with our data
eval_measures(goldlist, predictedlist)

	Precision	Recall		F1
high 	      0.510      0.383      0.438
low 	      0.788      0.862      0.823


In [ ]:
## cross-validation ##
# this function takes the number of folds, the feature sets
# it iterates over the folds, using different sections for training and testing in turn
#   it prints the accuracy for each fold and the average accuracy at the end
def cross_validation_accuracy(num_folds, featuresets):
    subset_size = int(len(featuresets)/num_folds)
    print('Each fold size:', subset_size)
    accuracy_list = []
    # iterate over the folds
    for i in range(num_folds):
        test_this_round = featuresets[(i*subset_size):][:subset_size]
        train_this_round = featuresets[:(i*subset_size)] + featuresets[((i+1)*subset_size):]
        # train using train_this_round
        classifier = nltk.NaiveBayesClassifier.train(train_this_round)
        # evaluate against test_this_round and save accuracy
        accuracy_this_round = nltk.classify.accuracy(classifier, test_this_round)
        print (i, accuracy_this_round)
        accuracy_list.append(accuracy_this_round)
    # find mean accuracy over all rounds
    print ('mean accuracy', sum(accuracy_list) / num_folds)

# perform the cross-validation on the featuresets with word features and generate accuracy
num_folds = 5
cross_validation_accuracy(num_folds, featuresets)


Each fold size: 6590
0 0.7309559939301973
1 0.7490136570561456
2 0.7959028831562974
3 0.678907435508346
4 0.7710166919575114
mean accuracy 0.7451593323216995


In [ ]:
# Demonstrate feature function: POS_features
# feature data set: comment_token and majorGiving

###  POS tag counts
# using the default pos tagger in NLTK (the Stanford tagger)
# print(all_words_list)
# print(nltk.pos_tag(all_words_list))

# this function takes a document list of words and returns a feature dictionary
# it runs the default pos tagger (the Stanford tagger) on the document
#   and counts FIVE types of pos tags to use as features
def POS_features(document, word_features):
    document_words = set(document)
    tagged_words = nltk.pos_tag(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    numNoun = 0
    numVerb = 0
    numAdj = 0
    numAdverb = 0
    numCardinal = 0
    for (word, tag) in tagged_words:
        if tag.startswith('N'): numNoun += 1
        if tag.startswith('V'): numVerb += 1
        if tag.startswith('J'): numAdj += 1
        if tag.startswith('R'): numAdverb += 1
        if tag.startswith('C'): numCardinal += 1
    features['nouns'] = numNoun
    features['verbs'] = numVerb
    features['adjectives'] = numAdj
    features['adverbs'] = numAdverb
    features['cardinal'] = numCardinal
    return features

# define feature sets using this function
POS_featuresets = [(POS_features(d, word_features), c) for (d, c) in CITRUSlist]
# number of features for document 0
print(len(POS_featuresets[0][0].keys()))

# the first sentence
print(CITRUSlist[0])
# the pos tag features for this sentence
print('num nouns', POS_featuresets[0][0]['nouns'])
print('num verbs', POS_featuresets[0][0]['verbs'])
print('num adjectives', POS_featuresets[0][0]['adjectives'])
print('num adverbs', POS_featuresets[0][0]['adverbs'])

# train and test the classifier
train_set, test_set = POS_featuresets[15000:], POS_featuresets[:15000]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)


2005
(['caleb', 'is', 'a', 'senior', 'at', 'unc', 'charlotte', 'and', 'is', 'interested', 'in', 'attending', 'law', 'school', 'in', 'addition', 'to', 'sucol', 'caleb', 'is', 'also', 'interested', 'in', 'the', 'charlotte', 'school', 'of', 'law'], 'low')
num nouns 8
num verbs 5
num adjectives 5
num adverbs 1


0.7280666666666666

In [ ]:
# show which features of classifier are most informative
classifier.show_most_informative_features(50)

Most Informative Features
           contains(lag) = True              low : high   =     45.6 : 1.0
  contains(disqualified) = True              low : high   =     29.6 : 1.0
                   verbs = 152              high : low    =     14.7 : 1.0
           contains(ths) = True              low : high   =     13.9 : 1.0
   contains(inclination) = True              low : high   =     12.4 : 1.0
              adjectives = 74               high : low    =      9.5 : 1.0
                cardinal = 77               high : low    =      9.4 : 1.0
                   nouns = 205              high : low    =      9.3 : 1.0
                cardinal = 64               high : low    =      8.9 : 1.0
         contains(molen) = True             high : low    =      8.8 : 1.0
           contains(ter) = True             high : low    =      8.8 : 1.0
           contains(wsm) = True             high : low    =      8.4 : 1.0
            contains(mg) = True              low : high   =      8.1 : 1.0

In [ ]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    	goldlist.append(label)
    	predictedlist.append(classifier.classify(features))
     
cm = nltk.ConfusionMatrix(goldlist, predictedlist)
print(cm.pretty_format(sort_by_count=True, truncate=9))

# or show the results as percentages
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=9))


     |         h |
     |    l    i |
     |    o    g |
     |    w    h |
-----+-----------+
 low |<9323>2597 |
high | 1482<1598>|
-----+-----------+
(row = reference; col = test)

     |             h |
     |      l      i |
     |      o      g |
     |      w      h |
-----+---------------+
 low | <62.2%> 17.3% |
high |   9.9% <10.7%>|
-----+---------------+
(row = reference; col = test)



In [ ]:
# Demonstrate feature function: document_features
# feature data set: comment_token and sentiment

CITRUSdf_st = df_pol[['comment_token','sentiment']]

CITRUSdf_st = CITRUSdf_st.query("sentiment!='neutral'")

CITRUSlist = list(CITRUSdf_st.itertuples(index=False, name=None))

# random.shuffle(CITRUSlist)

# get all words from all comments and put into a frequency distribution 
#   note lowercase, but no stemming or stopwords
all_words_list = [word for (sent,cat) in CITRUSlist for word in sent]
all_words = nltk.FreqDist(all_words_list)

# get the 2000 most frequently appearing keywords in the corpus 
word_items = all_words.most_common(2000)
word_features = [word for (word,count) in word_items]

# get features sets for a document, including keyword features and category feature
featuresets = [(document_features(d, word_features), c) for (d, c) in CITRUSlist]

# training using naive Baysian classifier, training set is approximately 90% of data 
train_set, test_set = featuresets[15000:], featuresets[:15000]
classifier = nltk.NaiveBayesClassifier.train(train_set)

# the accuracy result may vary since we randomized the documents 
# show which features of classifier are most informative
classifier.show_most_informative_features(50)

Most Informative Features
                V_thanks = True           positi : negati =     12.7 : 1.0
                V_amount = True           positi : negati =     11.4 : 1.0
                   V_rob = True           negati : positi =     10.2 : 1.0
              V_director = True           positi : negati =      9.0 : 1.0
           V_connections = True           positi : negati =      8.8 : 1.0
               V_enjoyed = True           positi : negati =      8.7 : 1.0
             V_impressed = True           positi : negati =      8.7 : 1.0
                   V_max = True           positi : negati =      8.7 : 1.0
                V_mentor = True           positi : negati =      8.6 : 1.0
                 V_enjoy = True           positi : negati =      8.5 : 1.0
                 V_loved = True           positi : negati =      8.3 : 1.0
               V_thanked = True           positi : negati =      8.2 : 1.0
                V_mutual = True           positi : negati =      8.2 : 1.0

In [ ]:
# evaluate the accuracy of the classifier
nltk.classify.accuracy(classifier, test_set)

In [ ]:
goldlist = []
predictedlist = []
for (features, label) in test_set:
    	goldlist.append(label)
    	predictedlist.append(classifier.classify(features))
     
cm = nltk.ConfusionMatrix(goldlist, predictedlist)
print(cm.pretty_format(sort_by_count=True, truncate=9))

# or show the results as percentages
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=9))

         |    p    n |
         |    o    e |
         |    s    g |
         |    i    a |
         |    t    t |
         |    i    i |
         |    v    v |
         |    e    e |
---------+-----------+
positive |<9631>4891 |
negative |  158 <320>|
---------+-----------+
(row = reference; col = test)

         |      p      n |
         |      o      e |
         |      s      g |
         |      i      a |
         |      t      t |
         |      i      i |
         |      v      v |
         |      e      e |
---------+---------------+
positive | <64.2%> 32.6% |
negative |   1.1%  <2.1%>|
---------+---------------+
(row = reference; col = test)



In [ ]:
## BACK TO POS TAGGING FOR FREQ DIST
# show how to flatten a list of tagged tokens
taggedtext_flat = [pair for sent in taggedText for pair in sent]
print(taggedtext_flat)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
tag_fd = nltk.FreqDist(tag for (word, tag) in taggedtext_flat)
print(tag_fd.keys(), '\n')


for tag,freq in tag_fd.most_common():
    print (tag, freq)

dict_keys(['NN', 'VBZ', 'DT', 'JJ', 'IN', 'CC', 'VBG', 'TO', 'VB', 'RB', 'VBD', 'PRP', 'PRP$', 'NNS', 'VBN', 'CD', 'EX', 'VBP', 'MD', 'RP', 'PDT', 'WP', 'JJS', 'WRB', 'WP$', 'WDT', 'RBR', 'JJR', 'RBS', 'FW', 'UH', 'NNP', '$', 'NNPS', 'POS', "''", 'SYM', 'LS']) 

NN 1206639
IN 654793
DT 401871
JJ 400516
PRP 312085
VBD 273038
VB 261844
NNS 228276
CC 215214
RB 214463
TO 188460
VBZ 152133
PRP$ 131118
VBG 120777
CD 104842
VBN 97067
VBP 93109
MD 74620
RP 28026
WRB 22357
WP 15422
WDT 15346
JJR 13028
JJS 7505
RBR 6450
EX 5208
PDT 2254
FW 2063
NNP 1871
RBS 1769
$ 675
UH 178
WP$ 152
POS 32
NNPS 19
SYM 7
'' 4
LS 1


In [ ]:
%%shell
jupyter nbconvert --to html /content/IST_664_Final_Project.ipynb

[NbConvertApp] Converting notebook /content/IST_664_Final_Project.ipynb to html
[NbConvertApp] Writing 498473 bytes to /content/IST_664_Final_Project.html
